In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mortuary temple of Hatshepsut/image_0_3467.jpg
/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mortuary temple of Hatshepsut/image_0_2949.jpg
/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mortuary temple of Hatshepsut/image_0_5172.jpg
/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mortuary temple of Hatshepsut/image_0_1809.jpg
/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mortuary temple of Hatshepsut/image_0_2713.jpg
/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mortuary temple of Hatshepsut/image_0_2704.jpg
/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mortuary temple of Hatshepsut/image_0_2928.jpg
/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mortuary temple of Hatshepsut/image_0_3726.jpg
/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mortuary temple of Hatshepsut/image_0_3754.jpg
/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mortuary temple of Hatshepsut/image_0_6143.jpg
/kaggle/input/egy-tour-data/Final_Aug/Final_Aug/Mo

In [2]:
! pip install livelossplot


# Import Librarys

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Dense ,Flatten , InputLayer  ,BatchNormalization
from keras.src.engine.training import optimizer
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


2024-02-26 16:37:30.766916: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 16:37:30.767013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 16:37:30.909244: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load Data 

In [4]:
train_dir = '/kaggle/input/smart-tour-data/Final_Aug/Final_Aug'

In [5]:
test_dir = '/kaggle/input/smart-tour-data/test'

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [7]:
train_data = train_generator.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=True,
    subset='training',
    seed=42
)

validation_data = train_generator.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=True,
    subset='validation',
    seed=42
)

test_data = test_generator.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=1,
    shuffle=False,
    seed=42
)


Found 4343 images belonging to 5 classes.
Found 1085 images belonging to 5 classes.
Found 120 images belonging to 5 classes.


In [8]:
# Get the class indices
train_indices = train_data.class_indices

print("train Indices:", train_indices)

# Get the class indices
test_indices = test_data.class_indices

print("test Indices:", test_indices)
# Get the class indices
val_indices = validation_data.class_indices

print("val Indices:", val_indices)

train Indices: {'Abdeen Palace Museum': 0, 'Abu Simbel': 1, 'Mortuary temple of Hatshepsut': 2, 'Qasr El Nil Bridge': 3, 'The Giza Pyramids': 4}
test Indices: {'Abdeen Palace Museum': 0, 'Abu Simbel': 1, 'Mortuary temple of Hatshepsut': 2, 'Qasr El Nil Bridge': 3, 'The Giza Pyramids': 4}
val Indices: {'Abdeen Palace Museum': 0, 'Abu Simbel': 1, 'Mortuary temple of Hatshepsut': 2, 'Qasr El Nil Bridge': 3, 'The Giza Pyramids': 4}


In [9]:
print("Number of Samples:", train_data.n)
print("Batch Size:", train_data.batch_size)

Number of Samples: 4343
Batch Size: 64


# Model 

In [10]:
mobilenet_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    alpha=1.0
)

9406464/9406464 [==============================] - 0s 0us/step


In [25]:
for layer in mobilenet_model.layers:
    layer.trainable = False


x=mobilenet_model.output


x = Flatten()(x)
x = Dense(512, activation='relu',kernel_regularizer=l2(0.01))(x)
x = Dense(1024, activation='relu',kernel_regularizer=l2(0.01))(x
x = Dense(512, activation='relu',kernel_regularizer=l2(0.01))(x)

x = Dense(5, activation='softmax')(x)

In [26]:
mobilenet_model = Model(inputs=mobilenet_model.input , outputs = x)

In [ ]:
print(mobilenet_model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']      

In [19]:
mobilenet_model.compile(optimizer = 'adam' ,loss = 'categorical_crossentropy', metrics = 'accuracy')

In [20]:
from livelossplot.inputs.keras import PlotLossesCallback
from keras.callbacks import ModelCheckpoint


plot_loss = PlotLossesCallback()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='best_weights.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=50,
                           restore_best_weights=True,
                           mode='min')

In [23]:
BATCH_SIZE = 32
n_steps = train_data.samples // BATCH_SIZE
n_val_steps = validation_data.samples // BATCH_SIZE
n_epochs = 100

In [ ]:
history = mobilenet_model.fit(train_data ,
                            validation_data = validation_data,
                            batch_size = BATCH_SIZE ,
                            epochs=n_epochs,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks = [early_stop ,tl_checkpoint_1, plot_loss]
                            )

In [19]:
evl = mobilenet_model.evaluate(test_data)

120/120 [==============================] - 2s 17ms/step - loss: 0.1224 - accuracy: 0.9750


In [21]:
mobilenet_model.load_weights('best_weights.hdf5')

true_classes = test_data.classes
class_indices = train_data.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

model_pred = mobilenet_model.predict(test_data)
model_pred_classes = np.argmax(model_pred ,axis=1)

120/120 [==============================] - 3s 15ms/step


In [22]:
from sklearn.metrics import accuracy_score

model_acc = accuracy_score(true_classes ,model_pred_classes)

print ('Mobilenet Model Accuracy :{:.2f}%'.format(model_acc * 100))

Mobilenet Model Accuracy :97.50%


In [23]:
mobilenet_model.save('Model.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
load_model = tf.keras.models.load_model("/kaggle/working/Model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(load_model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)